# 第4回：AIの社会実装（Webアプリ開発）

## 1. 目的
- 学習したモデルを保存し、ブラウザから利用できるWebアプリを作成する。
- Streamlitを用いて、直感的なUIを構築する。

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os

# リポジトリをクローン
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q

sys.path.insert(0, '/tmp/MedMNIST-Exercise')

# モジュールをインポート
sys.modules.pop('exercise_logic', None)
import exercise_logic

exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")

# 1. 最終モデルの保存（第1回〜第3回で作成したいずれかのモデルを想定）
# 注意: このセルを実行する前に、第1回〜第3回のいずれかで学習したモデルを変数 `model` として定義しておく必要があります。
# 例: 第1回のセッションで学習したモデルを使用する場合
#     (x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('pathmnist')
#     model = exercise_logic.build_model(input_shape=(28, 28, 3), num_classes=len(info['label']), model_type='simple')
#     model.fit(x_train, y_train, epochs=10, validation_split=0.1, batch_size=128)
#     その後、このセルで model.save() を実行

# モデルが既に定義されている場合のみ保存を実行
if 'model' in globals():
    model.save('medmnist_final_model.h5')
    print("モデルを 'medmnist_final_model.h5' として保存しました。")
else:
    print("警告: 変数 'model' が定義されていません。")
    print("第1回〜第3回のいずれかで学習したモデルを定義してから、このセルを実行してください。")

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image

st.set_page_config(page_title="Medical AI Assistant", page_icon="🩺")
st.title("🩺 医用画像診断アシスタント")

# モデルの読み込み
@st.cache_resource
def load_model():
    return tf.keras.models.load_model('medmnist_final_model.h5')

model = load_model()

uploaded_file = st.file_uploader("画像をアップロードしてください (28x28)", type=["jpg", "png", "jpeg"])

if uploaded_file:
    image = Image.open(uploaded_file).convert('RGB').resize((28, 28))
    st.image(image, caption='Uploaded Image', width=150)
    
    img_array = np.array(image).astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    preds = model.predict(img_array)[0]
    st.subheader(f"最有力候補の確信度: {np.max(preds):.2%}")
    st.bar_chart(preds)

## アプリの起動
以下のセルを実行して表示されるURLにアクセスしてください。
※Google Colabの場合、localtunnel等のトンネルツールが必要です。

In [ ]:
# ローカル環境（Cursor/VS Code）の場合：ターミナルで実行
# streamlit run app.py

# Colab等の場合：
!streamlit run app.py & npx localtunnel --port 8501